# First let's explore the model to understand what we have to build

In [1]:
from transformers import GPT2LMHeadModel #huggingface library

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torchvision/image.so, 6): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torchvision/image.so
  Expected in: /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torch/lib/libtorch_cpu.dylib
 in /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torchvision/image.so
  warn(f"Failed to load image Python extension: {e}")


In [2]:
model_hf = GPT2LMHeadModel.from_pretrained("gpt2") # 124M
sd_hf = model_hf.state_dict()

for k, v in sd_hf.items():
    print(k, v.shape)

transformer.wte.weight torch.Size([50257, 768])
transformer.wpe.weight torch.Size([1024, 768])
transformer.h.0.ln_1.weight torch.Size([768])
transformer.h.0.ln_1.bias torch.Size([768])
transformer.h.0.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias torch.Size([2304])
transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias torch.Size([768])
transformer.h.0.ln_2.weight torch.Size([768])
transformer.h.0.ln_2.bias torch.Size([768])
transformer.h.0.mlp.c_fc.weight torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias torch.Size([3072])
transformer.h.0.mlp.c_proj.weight torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias torch.Size([768])
transformer.h.1.ln_1.weight torch.Size([768])
transformer.h.1.ln_1.bias torch.Size([768])
transformer.h.1.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.1.attn.c_attn.bias torch.Size([2304])
transformer.h.1.attn.c_proj.weight torch.Size([768, 768])
transformer.h.1.attn.c_proj.bias 

In [3]:
sd_hf["transformer.wpe.weight"].view(-1)[:20]


tensor([-0.0188, -0.1974,  0.0040,  0.0113,  0.0638, -0.1050,  0.0369, -0.1680,
        -0.0491, -0.0565, -0.0025,  0.0135, -0.0042,  0.0151,  0.0166, -0.1381,
        -0.0063, -0.0461,  0.0267, -0.2042])

# Test our Model

In [3]:
import gpt2
import torch
# Inizializza il modello e input fittizio
config = gpt2.GPT2Config()
model = gpt2.GPT2(config)

# Input: batch di 2 sequenze lunghe 10, token random tra 0 e vocab_size
x = torch.randint(0, config.vocab_size, (2, 10))

# Forward pass
logits = model(x)

print("Logits :", logits[0].shape) # (2, 10, vocab_size)


Logits : torch.Size([2, 10, 65])


## Toy Example

In [4]:
# Dizionario toy per esempio char-level
import string

# Caratteri validi (simile a quello usato nei mini GPT)
chars = string.ascii_lowercase + " .,!?'"  # 26 lettere + punteggiatura + spazio
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}
vocab_size = len(stoi)

# Aggiorniamo la config per usare il vocab giusto
config.vocab_size = vocab_size
model = gpt2.GPT2(config)


In [5]:
def encode(text):
    return [stoi[c] for c in text if c in stoi]

def decode(tokens):
    return ''.join([itos[i] for i in tokens])


In [7]:
input_text = "hello my nam"
input_ids = encode(input_text)
x = torch.tensor([input_ids], dtype=torch.long)  # shape (1, T)

with torch.no_grad():
    logits = model(x)  # shape: (1, T, vocab_size)

# Estrai i logits dell'ultimo token
last_logits = logits[0][0, -1]  # shape: (vocab_size,)

# Prendi il token con la probabilità più alta
predicted_token_id = torch.argmax(last_logits).item()
predicted_char = itos[predicted_token_id]

print(f"Input: '{input_text}'")
print(f"Predicted next char: '{predicted_char}'")


Input: 'hello my nam'
Predicted next char: 'c'
